In [20]:
import pandas as pd
from glob import glob
import os as os
import sqlite3 as lite
con = lite.connect("solved")

In [21]:
con.execute("create Table IF NOT EXISTS PRODUCT (PRODUCTID NOT NULL, PRODUCTNAME TEXT NOT NULL, SupplierID TEXT NOT NULL, CategoryID INTEGER NOT NULL, Unit TEXT NOT NULL, Price INTEGER NOT NULL)")

In [22]:
PRODUCT = [
    (1, 'Chais', 1, 1, '10 boxes x 20 bags', 18.0),
    (2, 'Chang', 1, 1, '24 - 12 oz bottles', 19.0),
    (3, 'Aniseed Syrup', 1, 2, '12 - 550 ml bottles', 10.0),
    (4, "Chef Anton's Cajun Seasoning", 2, 2, '48 - 6 oz jars', 22.0),
    (5, "Chef Anton's Gumbo Mix", 2, 2, '36 boxes', 21.35),
    (6, "Grandma's Boysenberry Spread", 3, 2, '12 - 8 oz jars', 25.0),
    (7, "Uncle Bob's Organic Dried Pears", 3, 7, '12 - 1 lb pkgs.', 30.0),
    (8, "Northwoods Cranberry Sauce", 3, 2, '12 - 12 oz jars', 40.0),
    (9, "Mishi Kobe Niku", 4, 6, '18 - 500 g pkgs.', 97.0)
]
query = """
  INSERT INTO PRODUCT
  VALUES(?,?,?,?,?,?)
"""

con.executemany(query, PRODUCT)
pd.read_sql("Select * from PRODUCT",con)

,PRODUCTID,PRODUCTNAME,SupplierID,CategoryID,Unit,Price
0,1,Chais,1,1,10 boxes x 20 bags,18.00
1,2,Chang,1,1,24 - 12 oz bottles,19.00
2,3,Aniseed Syrup,1,2,12 - 550 ml bottles,10.00
3,4,Chef Anton's Cajun Seasoning,2,2,48 - 6 oz jars,22.00
4,5,Chef Anton's Gumbo Mix,2,2,36 boxes,21.35
5,6,Grandma's Boysenberry Spread,3,2,12 - 8 oz jars,25.00
6,7,Uncle Bob's Organic Dried Pears,3,7,12 - 1 lb pkgs.,30.00
7,8,Northwoods Cranberry Sauce,3,2,12 - 12 oz jars,40.00
8,9,Mishi Kobe Niku,4,6,18 - 500 g pkgs.,97.00


In [36]:
con.execute("CREATE TABLE IF NOT EXISTS OrderDetails(OrderDetailID INTEGER PRIMARY KEY, OrderID INTEGER NOT NULL,ProductID INTEGER NOT NULL,Quantity INTEGER NOT NULL)")

In [37]:
OrderDetails= [
    (1, 10248, 2, 12),
    (2, 10248, 4, 10),
    (3, 10248, 5, 5),
    (4, 10249, 2, 9),
    (5, 10249, 2, 40),
    (6, 10250, 2, 10),
    (7, 10250, 6, 35),
    (8, 10250, 7, 15),
    (9, 10251, 8, 6),
    (10, 10251, 20, 15)

]
query = """
  INSERT INTO OrderDetails
  VALUES(?,?,?,?)
"""

con.executemany(query, OrderDetails)
pd.read_sql("Select * from OrderDetails",con)

,OrderDetailID,OrderID,ProductID,Quantity
0,1,10248,2,12
1,2,10248,4,10
2,3,10248,5,5
3,4,10249,2,9
4,5,10249,2,40
5,6,10250,2,10
6,7,10250,6,35
7,8,10250,7,15
8,9,10251,8,6
9,10,10251,20,15


1.write a query product name group by their orderID

In [91]:
pd.read_sql( """
    SELECT OrderID, GROUP_CONCAT(PRODUCT.PRODUCTNAME) AS ProductNames
    FROM OrderDetails
    JOIN PRODUCT ON OrderDetails.ProductID = PRODUCT.PRODUCTID
    GROUP BY OrderID
""" , con)



,OrderID,ProductNames


2.write a query to select order details whose orderID's are same

In [44]:
pd.read_sql("""
    SELECT *
    FROM OrderDetails
    WHERE OrderID IN (
        SELECT OrderID
        FROM OrderDetails
        GROUP BY OrderID
        HAVING COUNT(*) > 1
    )
""", con)




,OrderDetailID,OrderID,ProductID,Quantity


3.write a query to select product name whose oredr quantity is grater than niminum quantity

In [67]:
pd.read_sql("""
    SELECT PRODUCT.PRODUCTNAME
    FROM OrderDetails
    JOIN PRODUCT ON OrderDetails.ProductID = PRODUCT.PRODUCTID
    WHERE OrderDetails.Quantity > (SELECT MIN(quantity) FROM OrderDetails)
    GROUP BY PRODUCT.PRODUCTNAME
""", con)


,PRODUCTNAME


4. write a quary to select product name whose order quantity is less than the average quantity

In [69]:
pd.read_sql("""
    SELECT PRODUCT.PRODUCTNAME
    FROM OrderDetails
    JOIN PRODUCT ON OrderDetails.ProductID = PRODUCT.PRODUCTID
    GROUP BY PRODUCT.PRODUCTNAME
    HAVING AVG(OrderDetails.Quantity) > OrderDetails.Quantity
""", con)


,PRODUCTNAME


5. write a query to create view for selecting product product name who doesn't have any order quantity

In [74]:
con.execute("DROP VIEW IF EXISTS ProductsWithNoOrders")

con.execute("""
    CREATE VIEW ProductsWithNoOrders AS
    SELECT PRODUCT.PRODUCTNAME
    FROM PRODUCT
    LEFT JOIN OrderDetails ON PRODUCT.PRODUCTID = OrderDetails.ProductID
    WHERE OrderDetails.Quantity IS NULL
""")


6.write a quary to select product name which has maximum and minimum price group their categoryID

In [89]:
pd.read_sql("""
    SELECT categoryID,
           MAX(productname) AS max_price_product,
           MIN(productname) AS min_price_product
    FROM product
    GROUP BY categoryID
""", con)


,CategoryID,max_price_product,min_price_product
